# Scraper
### Notebook con el scraper para armar el dataset de noticias de Venezuela
## Features iniciales
### titulo: titulo de la noticia
### texto: cuerpo de la noticia
### fecha: fecha de publicación de a noticia
### link: dirección url de la noticia

In [7]:
import feedparser as fp
import json
import newspaper
import datetime 
import dateutil.parser
from datetime import date, timedelta
from newspaper import Article
from time import mktime
from datetime import datetime

In [8]:
def getDateTimeFromISO8601String(iso): 
    f = dateutil.parser.parse(iso) 
    return f

In [9]:
# Seteo del límite de la cantidad de noticias por sitio web
LIMIT = 100

data = {}
data['newspapers'] = {}

In [10]:
# Cargar el JSON de archivos con las urls a los diarios en línea
with open('NewsPapers.json') as data_file:
    companies = json.load(data_file)

In [6]:
count = 1

# Iteración por cada diario electrónico
for company, value in companies.items():
    print("Construyendo el sitio para ", company)
    paper = newspaper.build(value['link'], memoize_articles=False)
    newsPaper = {
        "link": value['link'],
        "articles": []
    }
    print("Sitio construido para ", company)
    noneTypeCount = 0
    for content in paper.articles:
        if count > LIMIT:
            break
        try:
            content.download()
            content.parse()
        except Exception as e:
            print(e)
            print("continuing...")
            continue
        # Se saltará el artículo que no posea fecha de publicación.
        # Después de tener 10 artículos sin fecha, el diario electrónico será saltado.
        if content.publish_date is None:
            print(" Article has date of type None...")
            noneTypeCount = noneTypeCount + 1
            if noneTypeCount > 10:
                print("Too many noneType dates, aborting...")
                noneTypeCount = 0
                break
            continue
        else:
            #print(content.title)
            #print(content.text)
            #print(content.url)
            #print(content.publish_date.isoformat())
            #fecha = getDateTimeFromISO8601String(content.publish_date.isoformat())
            #fecha_desde = datetime(2018, 1, 1, 0, 0, 0)
            #fecha_hasta = datetime(2018,12,31,23,59,59)
            palabras = ["Maduro","Venezuela"]
            for palabra in palabras:
                if palabra in content.title:
                    #if fecha <= fecha_hasta and fecha >= fecha_desde:
                    article = {}
                    article['titulo'] = content.title
                    article['texto'] = content.text
                    article['link'] = content.url
                    article['fecha'] = content.publish_date.isoformat()
                    newsPaper['articles'].append(article)
                    print("Van ",count, " artículos bajados de ", company, " usando el sitio de url: ", content.url)
                    count = count + 1
                    noneTypeCount = 0
                    data['newspapers'][company] = newsPaper
                    try:
                        with open('scraped_articles.json', 'w') as outfile:
                            json.dump(data, outfile)
                    except Exception as e: print(e)

Construyendo el sitio para  cnn
Sitio construido para  cnn
 Article has date of type None...
 Article has date of type None...
 Article has date of type None...
 Article has date of type None...
 Article has date of type None...
Article `download()` failed with 404 Client Error: Not Found for url: https://fox13now.com/2019/02/15/herriman-residents-recall-close-call-with-mountain-lion-that-may-have-caused-lockout-at-schools/ on URL https://fox13now.com/2019/02/15/herriman-residents-recall-close-call-with-mountain-lion-that-may-have-caused-lockout-at-schools/
continuing...
 Article has date of type None...
 Article has date of type None...
 Article has date of type None...
 Article has date of type None...
 Article has date of type None...
 Article has date of type None...
Too many noneType dates, aborting...
Construyendo el sitio para  bbc
Sitio construido para  bbc
 Article has date of type None...
 Article has date of type None...
 Article has date of type None...
 Article has date of

KeyboardInterrupt: 

Van  4  artículos bajados de  universo  usando el sitio de url:  https://www.eluniverso.com/noticias/2019/02/16/nota/7191752/juan-guaido-organiza-voluntarios-recibir-ayuda-humanitaria-maduro
Van  5  artículos bajados de  universo  usando el sitio de url:  https://www.eluniverso.com/noticias/2019/02/15/nota/7190921/oea-propone-reformas-urgentes-profundas-propiciar-elecciones


KeyboardInterrupt: 

In [ ]:
import json
import os

data = {}
data['nombre'] = 'Jose'
data['edad'] = '15'
data['nacionalidad'] = 'Mex'

dir = 'C:/Users/User/Thalía/Espol/2018-2S/Minería/noticiasVenezuela'
file_name = "data.json"

with open(os.path.join(dir, file_name), 'w') as file:
    json.dump(data, file)